# scVI for normalized_gene_expression

scVI is run here to obtain a more smoothed gene expression landscape. This is mostly done because for cells with 0 expression in some genes correlations of pull to these driver genes does not work well. 

In [1]:
import numpy as np
import anndata
import scanpy as sc
import scvi

Global seed set to 0
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/mgander/miniconda3/envs/mc/lib/python3.10/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
!pip list

Package                           Version
--------------------------------- --------------------
absl-py                           1.2.0
anndata                           0.8.0
anyio                             3.6.1
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
asttokens                         2.0.5
astunparse                        1.6.3
attrs                             22.1.0
Babel                             2.10.3
backcall                          0.2.0
backports.functools-lru-cache     1.6.4
beautifulsoup4                    4.11.1
bleach                            5.0.1
bokeh                             2.4.3
brotlipy                          0.7.0
CacheControl                      0.12.11
cached-property                   1.5.2
cachetools                        5.2.0
cachy                             0.3.0
certifi                           2022.12.7
cffi                              1.15.1
charset-normalizer                2.1.0
chex       

toml                              0.10.2
tomli                             2.0.1
tomlkit                           0.11.5
toolz                             0.12.0
torch                             2.0.0
torchvision                       0.15.1
tornado                           6.2
tqdm                              4.64.0
traitlets                         5.3.0
triton                            2.0.0
twine                             4.0.1
typing_extensions                 4.3.0
umap-learn                        0.5.3
urllib3                           1.26.11
virtualenv                        20.16.5
wcwidth                           0.2.5
webencodings                      0.5.1
websocket-client                  1.3.3
Werkzeug                          2.2.2
wheel                             0.37.1
widgetsnbextension                4.0.2
wrapt                             1.14.1
xarray                            2023.1.0
zipp                              3.8.0


In [3]:
Path="/home/mgander/Unrelated/moscotTime_Reproducibility/Data"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']

# Running scVI over all time points of interest

In [ ]:
number_of_hvgs=3000

for i in [7,9,10,16]:
    ts0=ts[i]
    ts1=ts[i+1]
    print(ts0)
    # Prepare the data set
    adata=anndata.read(f'{Path}/anndatas/adata_{ts0}.h5ad')
    adata=adata.raw.to_adata()
    adata.layers["counts"] = adata.X.copy()
    sc.pp.highly_variable_genes(adata, n_top_genes=number_of_hvgs, subset=True, layer="counts", flavor="seurat_v3")
    
    # Run scVI
    scvi.model.SCVI.setup_anndata(adata, layer="counts")
    model = scvi.model.SCVI(adata)
    model.train()
    adata.obsm["X_scVI"]=model.get_latent_representation()
    adata.layers["scvi_normalized"] = model.get_normalized_expression(library_size=10e4)
    adata.write(f'{Path}/anndatas/scvi_adata_{ts0}.h5ad', compression='gzip')

E7.0


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 30/400:   7%|█████                                                                 | 29/400 [00:27<05:00,  1.23it/s, loss=1.59e+03, v_num=1]

In [19]:
number_of_hvgs=3000

for i in [11]:
    ts0=ts[i]
    ts1=ts[i+1]
    print(ts0)
    # Prepare the data set
    adata=anndata.read(f'{Path}/anndatas/Comb_anndatas/adata_{ts0}_{ts1}.h5ad')
    adata=adata[adata.obs['day']==min(set(adata.obs['day']))].copy()
    avar=adata.var.copy()
    adata=adata.raw.to_adata()
    adata.var=avar.copy()
    adata.layers["counts"] = adata.X.copy()
    sc.pp.highly_variable_genes(adata, n_top_genes=number_of_hvgs, subset=True, layer="counts", flavor="seurat_v3")


    # Run scVI
    scvi.model.SCVI.setup_anndata(adata, layer="counts")
    model = scvi.model.SCVI(adata)
    model.train()
    adata.obsm["X_scVI"]=model.get_latent_representation()
    adata.layers["scvi_normalized"] = model.get_normalized_expression(library_size=10e4)
    adata.write(f'{Path}/anndatas/scVI_anndatas/scvi_adata_{ts0}.h5ad', compression='gzip')

E8.0


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 400/400: 100%|██████████████████████████████████████████████████████████████████████████████████| 400/400 [27:45<00:00,  4.16s/it, loss=1.37e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████████████████████████████████████████████████████████████████████████████| 400/400 [27:45<00:00,  4.16s/it, loss=1.37e+03, v_num=1]
